In [ ]:
import pandas as pd 
df = pd.read_parquet('../scripts/df_merged.parquet', engine='pyarrow')

In [ ]:
df[df['objectId'] == 'ZTF17aaaeclk'][['mjd', 'fid', 'dc_flux', 'dc_sigflux', 'nr_flux', 'nr_sigflux', 'source']]

In [ ]:
unique_ids = df['objectId'].unique().tolist()
unique_ids

In [ ]:
for i in unique_ids:
    print(df[df['objectId'] == i][['mjd', 'fid', 'dc_flux', 'dc_sigflux', 'nr_flux', 'nr_sigflux', 'source']])